<a href="https://www.kaggle.com/code/pratyushraj07/train?scriptVersionId=136783025" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
x_train1 = np.load('/kaggle/input/x-train/x_train1.npy')
y_train1 = np.load('/kaggle/input/y-train/y_train1.npy')

In [ ]:
x_train1

In [ ]:
from sklearn.model_selection import train_test_split
x_sptrain, x_spval, y_sptrain, y_spval = train_test_split(
    x_train1, y_train1, 
    random_state=42
)


In [ ]:
import os
from matplotlib import pyplot as plt
import cv2
import gc
import tensorflow as tf
from tqdm import tqdm
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from keras.applications import DenseNet121,DenseNet169,DenseNet201

In [ ]:
# del x_train1,y_train1
# gc.collect()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
def create_datagen():
    return ImageDataGenerator(
        horizontal_flip=True,   # randomly flip images
        vertical_flip=True,     # randomly flip images
        rotation_range=20 ,      # Degree range for random rotations
        zoom_range=0.1
    )

# Using original generator
data_generator = create_datagen().flow(x_sptrain, y_sptrain,batch_size=16,seed=42)
print("Image data augmentated ...")

In [ ]:
# import tensorflow as tf
# import keras.backend as K
# def f1(y_true, y_pred):
#     y_pred = K.round(y_pred)
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

#     p = tp / (tp + fp + K.epsilon())
#     r = tp / (tp + fn + K.epsilon())

#     f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#     return K.mean(f1)

In [ ]:
import tensorflow as tf
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


In [ ]:
effnet = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(256,256,3)
)

In [ ]:
effnet = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(256,256,3)
)

In [ ]:
model = Sequential()
model.add(effnet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dense(5, activation='sigmoid'))

model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=0.00005),
        metrics=['accuracy'],
)

In [ ]:
# from keras.callbacks import Callback,ModelCheckpoint
# from sklearn.metrics import cohen_kappa_score
# import json
# class KappaMetrics(Callback):
#     i=0
#     def _init_(self, val_data, batch_size = 16):
#         super()._init_()
#         self.validation_data = val_data
#         self.batch_size = batch_size

#     def on_train_begin(self, logs={}):
#         self.val_kappas = []
        
#     def on_epoch_end(self, epoch, logs={}):
#         X_val, y_val = self.validation_data[:2]
#         y_val = y_val.sum(axis=1) - 1
        
#         y_pred = self.model.predict(X_val) > 0.5
#         y_pred = y_pred.astype(int).sum(axis=1) - 1

#         _val_kappa = cohen_kappa_score(
#             y_val,
#             y_pred, 
#             weights='quadratic'
#         )

#         self.val_kappas.append(_val_kappa)

#         print(f"Epoch: {epoch+1} val_kappa: {_val_kappa:.4f}")
        
#         if _val_kappa == max(self.val_kappas):
#             print("Validation Kappa has improved. Saving model. - "+str(KappaMetrics.i))
#             model_json = self.model.to_json()
#             with open("model_in_json"+str(KappaMetrics.i)+".json", "w") as json_file:
#                 json.dump(model_json, json_file)
#             self.model.save_weights("model_weights"+str(KappaMetrics.i)+".h5")
#             KappaMetrics.i+=1
#         return
    
# kappa_score = KappaMetrics()

In [ ]:
!pip install --upgrade tensorflow


In [ ]:
import tensorflow as tf

print(tf.__version__)


In [ ]:
print(data_generator)

In [ ]:
print("x_spval shape:", x_spval.shape)
print("y_spval shape:", y_spval.shape)
print("x_spval dtype:", x_spval.dtype)
print("y_spval dtype:", y_spval.dtype)


In [ ]:
print(x_spval)

In [ ]:
from keras.callbacks import Callback,ModelCheckpoint
from sklearn.metrics import cohen_kappa_score
import json

class KappaMetrics(Callback):
    i=0
    def _init_(self, val_data, batch_size = 8):
        super()._init_()
        self.validation_data = val_data
        self.batch_size = batch_size
        
    def on_train_begin(self, logs={}):
        self.val_kappas = []
        
    def on_epoch_end(self, epoch, logs={}):
        batches = len(self.validation_data)
        val_pred = np.zeros((batches * self.batch_size, 1))
        val_true = np.zeros((batches * self.batch_size))

        for batch in range(batches):
            x_val_batch, y_val_batch = self.validation_data[batch]
            start_idx = batch * self.batch_size
            end_idx = (batch + 1) * self.batch_size

            val_pred[start_idx:end_idx] = np.argmax(self.model.predict(x_val_batch), axis=1, keepdims=True)
            val_true[start_idx:end_idx] = y_val_batch.sum(axis = 1) - 1

        _val_kappa = cohen_kappa_score(val_true, val_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)

        print(f"Epoch: {epoch+1} val_kappa: {_val_kappa:.4f}")

        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model. - " + str(KappaMetrics.i))
            model_json = self.model.to_json()
            with open("model_in_json" + str(KappaMetrics.i) + ".json", "w") as json_file:
                json.dump(model_json, json_file)
            self.model.save_weights("model_weights" + str(KappaMetrics.i) + ".h5")
            KappaMetrics.i += 1
        return

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import cohen_kappa_score
import json

class KappaMetrics(Callback):
    i = 0

    def __init__(self, val_data, batch_size=8):
        super().__init__()
        self.validation_data = val_data
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs=None):
        batches = len(self.validation_data)
        val_pred = np.zeros((batches * self.batch_size, 1))
        val_true = np.zeros((batches * self.batch_size))

        for batch in range(batches):
            
            x_val_batch, y_val_batch = next(self.validation_data)
            start_idx = batch * self.batch_size
            end_idx = start_idx + len(x_val_batch)

#             x_val_batch, y_val_batch = self.validation_data[batch]
#             start_idx = batch * self.batch_size
#             end_idx = (batch + 1) * self.batch_size

            val_pred[start_idx:end_idx] = np.argmax(self.model.predict(x_val_batch), axis=1, keepdims=True)
            val_true[start_idx:end_idx] = y_val_batch.sum(axis=1) - 1

        _val_kappa = cohen_kappa_score(val_true, val_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)

        print(f"Epoch: {epoch+1} val_kappa: {_val_kappa:.4f}")

        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model. - " + str(KappaMetrics.i))
            model_json = self.model.to_json()
            with open("model_in_json" + str(KappaMetrics.i) + ".json", "w") as json_file:
                json.dump(model_json, json_file)
            self.model.save_weights("model_weights" + str(KappaMetrics.i) + ".h5")
            KappaMetrics.i += 1


In [ ]:
val_datagen = ImageDataGenerator(rescale = 1/255.0)
val_data_generator = val_datagen.flow(x_spval, y_spval, batch_size=8, seed=42)
#print(val_data_generator._getitem_(1)[1].shape)

In [ ]:
kappa_score = KappaMetrics(val_data=val_data_generator)
history = model.fit(
    data_generator,
    steps_per_epoch=10,
    epochs=8,
    validation_data=val_data_generator,  # Use the validation data generator
    callbacks=[kappa_score]
)

In [ ]:
print(y_spval)

In [ ]:
history = model.fit(
    data_generator,
    steps_per_epoch=20,  # Use integer division instead of floating-point division
    epochs=8,
    validation_data=(x_spval, y_spval),
    callbacks=[kappa_score],
)


In [ ]:
# history = model.fit_generator(
#     data_generator,
#     steps_per_epoch= 3362/16,
#     epochs=8,
#     validation_data=(x_spval, y_spval),
#     callbacks=[kappa_score],
# )

In [ ]:
y=pd.read_csv('/kaggle/input/aptos2019-blindness-detection/test.csv')

In [ ]:
y.head()

In [ ]:
from keras.preprocessing import image

In [ ]:
size = (256,256)

In [ ]:
pred1 = []

In [ ]:
from PIL import Image

for i in y['id_code']:
    img = Image.open('/kaggle/input/aptos2019-blindness-detection/test_images/' + i + '.png')
    img = img.resize(size)
    img = np.array(img)
    img = img.reshape(1, 256, 256, 3)
    output = model.predict(img)
    pred1 = np.append(pred1, np.argmax(output[0]))


In [ ]:
pred = []
for j in pred1:
    pred.append(int(j))
print(pred)


In [ ]:
# for i in y['id_code']:
#     img=image.load_img(('/kaggle/input/aptos2019-blindness-detection/test_images'+i+".png"),target_size=size)
#     img=image.img_to_array(img)
#     img=img.reshape(1,128,128,3)
#     output=model.predict(img)
#     pred1=np.append(pred1,(np.argmax(output[0])))

In [ ]:
x = pd.DataFrame({'id_code':y['id_code'],'diagnosis': pred })
print(x)

In [ ]:
sub = x.to_csv('submission.csv')